In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
sns.set_theme()

In [3]:
def distance(p, q):
    return np.sqrt((p - q) @ (p - q))

In [4]:
#function for main algorithm
def k_nearest_neighbors(point, 
                        training_features, 
                        training_labels, 
                        k):
    # Create an empty list to store neighbors and distances
    neighbors = []
    
    for p, label in zip(training_features, training_labels):
        d = distance(point, p)
        temp_data = [p, label, d]
        neighbors.append(temp_data)
        
    neighbors.sort(key = lambda x : x[-1])
    
    return neighbors[:k]

In [5]:
#function for predicting target labels
def KNN_Predict(point, 
                training_features, 
                training_labels, 
                k, 
                regression = False):
    
    neighbors = k_nearest_neighbors(point, 
                                    training_features, 
                                    training_labels, 
                                    k)
    
    if regression == False:
        labels = [x[1] for x in neighbors]
        return max(labels, key = labels.count)
    
    else:
        return sum(x[1] for x in neighbors)/k

In [26]:
#recommand system
games = pd.read_csv('games.csv')
games.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,11/21/2008,True,False,False,Very Positive,84,2199,9.99,9.99,0,True
1,22364,BRINK: Agents of Change,8/3/2011,True,False,False,Positive,85,21,2.99,2.99,0,True
2,113020,Monaco: What's Yours Is Mine,4/24/2013,True,True,True,Very Positive,92,3722,14.99,14.99,0,True
3,226560,Escape Dead Island,11/18/2014,True,False,False,Mixed,61,873,14.99,14.99,0,True
4,249050,Dungeon of the ENDLESS™,10/27/2014,True,True,False,Very Positive,88,8784,11.99,11.99,0,True


In [27]:
#keep first 500 rows
games = games.head(500)
games = games.drop(columns=['date_release', 'app_id'], axis=1)
games = games.replace(True, 1)
games = games.replace(False, 0)
games = games.replace({'rating' : { 'Very Positive' : 7, 'Positive' : 6, 'Mixed' : 4, 'Mostly Negative': 3,
                                   'Mostly Positive': 5, 'Negative': 2, 'Overwhelmingly Negative': 0, 
                                   'Overwhelmingly Positive': 8, 'Very Negative': 1,}})
games.set_index("title", drop = False, inplace = True)
games

,title,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
title,,,,,,,,,,,
Prince of Persia: Warrior Within™,Prince of Persia: Warrior Within™,1,0,0,7,84,2199,9.99,9.99,0,1
BRINK: Agents of Change,BRINK: Agents of Change,1,0,0,6,85,21,2.99,2.99,0,1
Monaco: What's Yours Is Mine,Monaco: What's Yours Is Mine,1,1,1,7,92,3722,14.99,14.99,0,1
Escape Dead Island,Escape Dead Island,1,0,0,4,61,873,14.99,14.99,0,1
Dungeon of the ENDLESS™,Dungeon of the ENDLESS™,1,1,0,7,88,8784,11.99,11.99,0,1
...,...,...,...,...,...,...,...,...,...,...,...
SCP: Doki Doki Anomaly,SCP: Doki Doki Anomaly,1,0,0,7,92,64,5.99,5.99,0,1
Impostor Factory Soundtrack,Impostor Factory Soundtrack,0,0,0,6,100,34,4.99,4.99,0,1
PYRMD,PYRMD,1,0,0,5,73,15,7.99,7.99,0,1


In [28]:
def games_recommendation(games, df, k):
    categories = ['win', 'mac', 'linux', 'rating', 'positive_ratio', 'user_reviews', 'price_final', 
                  'price_original', 'discount', 'steam_deck']
    X = df[categories].to_numpy()
    y = df["title"].to_numpy()

    point = df[categories].loc[games].to_numpy()
    neighbors = k_nearest_neighbors(point, X, y, k+1)
    return [x[1] for x in neighbors[1:]]

In [29]:
recommendations = games_recommendation('The Supper', games, 3)
#Return top 3 games recommendation
for i, game in enumerate(recommendations):
    print(f"{i+1}. {game}")

Games Recommendation
1. DARQ: Complete Edition
2. Blood Trail
3. The Thrill of the Fight - VR Boxing
